In [1]:
import pandas as pd

import numpy as np

import holoviews as hv
import plotly.graph_objects as go
import plotly.express as pex

In [2]:
df_raw = pd.read_csv('data/goemotions.csv')

In [3]:
emotions_cols = ['admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']

In [4]:
emotion_categories = {'admiration': 'Positive', # Proposed simplified classification
                      'amusement':'Positive',
                      'anger':'Negative',
                      'annoyance':'Negative',
                      'approval':'Positive',
                      'caring':'Positive',
                      'confusion':'Neutral',
                      'curiosity':'Positive',
                      'desire':'Positive',
                      'disappointment':'Negative',
                      'disapproval':'Negative',
                      'disgust':'Negative',
                      'embarrassment':'Neutral',
                      'excitement':'Positive',
                      'fear':'Negative',
                      'gratitude':'Positive',
                      'grief':'Negative',
                      'joy':'Positive',
                      'love':'Positive',
                      'nervousness':'Neutral',
                      'optimism':'Positive',
                      'pride':'Positive',
                      'realization':'Neutral',
                      'relief':'Positive',
                      'remorse':'Negative',
                      'sadness':'Negative',
                      'surprise':'Neutral',
                      'neutral':'Neutral'
                       }

In [14]:
df = df_raw.groupby(['id', 'subreddit', 'text'])[emotions_cols].sum().reset_index() # Aggregates by post

In [6]:
# Only keeps emotion(s) with the most votes for each post
"""
df['max_val'] = df[emotions_cols].max(axis=1)
for col in emotions_cols:
    df[col] = df[col] - df['max_val']
del df['max_val']
df_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')
       .groupby(['id', 'subreddit'])['emotion']
       .apply(', '.join)
       .reset_index())
df_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()
"""

"\ndf['max_val'] = df[emotions_cols].max(axis=1)\nfor col in emotions_cols:\n    df[col] = df[col] - df['max_val']\ndel df['max_val']\ndf_new = (df.melt(['id', 'subreddit'], var_name='emotion').query('value >= 0')\n       .groupby(['id', 'subreddit'])['emotion']\n       .apply(', '.join)\n       .reset_index())\ndf_grouped = df_new.groupby(['subreddit', 'emotion']).count().reset_index()\n"

In [15]:
df.groupby(['subreddit']).count().sort_values('id')

,id,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
subreddit,,,,,,,,,,,,,,,,,,,,,
farcry,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
shieldbro,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Anticonsumption,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
SSBM,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
darknet,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vanderpumprules,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
AnimalsBeingBros,231,231,231,231,231,231,231,231,231,231,...,231,231,231,231,231,231,231,231,231,231
socialanxiety,232,232,232,232,232,232,232,232,232,232,...,232,232,232,232,232,232,232,232,232,232


In [20]:
subreddit_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'count'])
for emotion in emotions_cols:
    gp_df = df.groupby(['subreddit'])[emotion].sum()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'count':gp_df.values}
    subreddit_df = subreddit_df.append(pd.DataFrame(data = d), ignore_index=True)

In [9]:
#hv.extension('bokeh')
#hv.Sankey(subreddit_df.loc[lambda f: f['subreddit'] == 'socialanxiety'])

In [59]:
#subreddit_df.to_csv('subreddit_emotion.csv')

In [39]:
text_df = pd.DataFrame(columns = ['subreddit', 'emotion', 'text'])
for emotion in emotions_cols:
    df_long = df[df['text'].str.split().str.len().ge(4)] # Remove short posts
    df_sorted = df_long.sort_values(emotion, ascending=False)
    gp_df = df_sorted.groupby('subreddit').first()
    d = {'subreddit': gp_df.index, 'emotion': [emotion]*len(gp_df), 'text':gp_df['text']}
    text_df = text_df.append(pd.DataFrame(data = d), ignore_index=True)

In [44]:
full_df = pd.merge(text_df, subreddit_df, how='left', on=['subreddit', 'emotion'])

In [46]:
full_df.columns = ['source', 'target', 'text', 'value']

In [47]:
full_df.to_csv('subreddit_emotion_2.csv')

In [45]:
full_df

,subreddit,emotion,text,count
0,2meirl4meirl,admiration,Or embalmed! Green burial is the best burial,47
1,49ers,admiration,Can we please just call out defense next year ...,50
2,4PanelCringe,admiration,Seriously. [NAME] is a hero. I heard he was a ...,52
3,90DayFiance,admiration,maybe. Id praise her for it though. In a world...,76
4,90dayfianceuncensored,admiration,She could and should pursue modeling! It seeme...,43
...,...,...,...,...
13519,worldpolitics,neutral,The original video was debunked. Do you agree ...,156
13520,yesyesyesyesno,neutral,This is like 4 years old from vine,198
13521,youseeingthisshit,neutral,Imagine being upset over who someone loves.,198
13522,youtube,neutral,Naming comes from the heart,95
